<a href="https://colab.research.google.com/github/ransjnr/building-ai-agents-with-autogen/blob/main/Creating_sophisticated_agents_with_nested_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ag2
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.1 MB/s eta 0:00:00


In [ ]:
##import requried libraries

import os
from autogen import ConversableAgent, UserProxyAgent, AssistantAgent
from openai import AzureOpenAI
from google.colab import userdata


In [ ]:
endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
model_name = "o4-mini"
deployment = "o4-mini-deployment"

subscription_key = userdata.get("AZURE_OPENAI_KEY")
api_version = "2024-12-01-preview"

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": deployment,
            "base_url": endpoint,
            "api_type": "azure",
            "api_key": subscription_key,
            "api_version": api_version,
        }
    ]
}

###Writing Agent

In [ ]:
writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
                   "blogpost (with title) on given topics. You must polish your "
                   "writing based on the feedback you receive and give a refined "
                   "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

###Task

In [ ]:
task='''
      Write a concise but engaging blogpost about GPUs.
      Make sure the blogpost is within 100 words.
      '''

In [ ]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

[autogen.oai.client: 05-13 03:30:55] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [ ]:
print(reply)

Title: How GPUs Power the Future of Computing

In today’s digital age, Graphics Processing Units (GPUs) are the unsung heroes powering everything from lifelike video games to breakthrough AI research. Unlike traditional CPUs, GPUs excel at parallel processing, tackling thousands of tasks simultaneously—ideal for rendering complex visuals, crunching scientific data, and training neural networks. As industries demand higher performance and efficiency, GPUs evolve with specialized cores and smarter architectures. Whether you’re a gamer, a data scientist, or a content creator, embracing GPU technology means unlocking faster speeds, richer experiences, and unprecedented innovation.


###Simple review of the post

In [ ]:
reviewer = AssistantAgent(
    name = "Reviewer",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a reviewer. You review the work of "
                  "the writer and provide constructive "
                  "feedbackto help improve the quality of the content.",
)

In [ ]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):


      Write a concise but engaging blogpost about GPUs.
      Make sure the blogpost is within 100 words.
      

--------------------------------------------------------------------------------
[autogen.oai.client: 05-13 03:31:16] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Writer (to Reviewer):

Unleashing the Power of GPUs

Graphics processing units (GPUs) have transformed modern computing, accelerating everything from gaming to scientific simulations. Originally designed to render images, today’s GPUs deploy thousands of cores to tackle complex parallel tasks, powering AI, machine learning, and data analytics. Their massive throughput and energy efficiency make them indispensable in research labs, animation studios, and cryptocurrency mining. As developers optimize algorithms to harness GPU capabilities, we stand on the brink of breakthroughs in virtual reality, deep learning, and beyond. Embrace the GPU revolution and unlock limitless computational potential.

--------------------------------------------------------------------------------
[autogen.oai.client: 05-13 03:31:30] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pri

Reviewer (to Writer):

You’ve hit most of the marks: it’s short, clear and uses strong verbs (“accelerating,” “powering,” “unlock”) that keep the reader’s interest. A few thoughts to push it even further:

1. Lead with a hook  
   Instead of the title alone, try opening with a vivid snapshot or question (“Ever watched a 3D world come to life in an instant?”) to draw readers in immediately.

2. Define jargon sparingly  
   Words like “throughput” and “parallel tasks” might lose non-technical readers. Consider swapping in plain-English phrases (“raw speed,” “many calculations at once”) or briefly explaining them.

3. Add a concrete example  
   Mention a familiar GPU-powered game, a popular AI application, or a record-breaking benchmark to make the technology feel tangible.

4. Tighten the close  
   Your final line is strong, but you could make it more actionable—perhaps invite readers to try a GPU-powered demo or link to a beginner’s guide.

Overall it’s a compelling snapshot of GPUs’ 

Writer (to Reviewer):

Unleashing GPU Power

Ever watched a 3D world come to life in an instant? GPUs—specialized chips originally built for rendering graphics in games like Fortnite—now deliver raw speed by handling thousands of calculations at once. Beyond gaming, they power AI tools such as ChatGPT, accelerate scientific research, and drive real-time virtual reality. Thanks to their energy-efficient design, GPUs are a staple in animation studios, data centers, and crypto mining rigs. Ready to explore? Try a free GPU-powered demo or dive into our beginner’s guide to get started today.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (4363185a-7c50-4083-872c-ffe0f67acc5d): Maximum turns (2) reached


###Adding multi-level review using nested chats


In [ ]:
SEO_reviewer = AssistantAgent (
    name="SEO_reviewer",
    llm_config=llm_config,
    system_message="As an SEO expert, your role is to analyze and enhance content for optimal search engine performance. "
                   "Focus on providing actionable recommendations that boost rankings and drive organic traffic. "
                   "Limit your feedback to 3 key points, ensuring they are specific and directly applicable. "
                   "Start each review by introducing yourself as an SEO Reviewer.",
)

In [ ]:
grammatical_error_reviewer = AssistantAgent(
    name="GrammaticalErrorReviewer",
    llm_config=llm_config,
    system_message="As a grammar specialist, your task is to meticulously examine content "
                   "for grammatical errors, punctutation mistakes, and style inconsistencies. "
                   "Provide up to 3 key points addressing the most significant grammatical issues. "
                   "Ensure your feedback is specific, actionable, and includes examples where appropriate. "
                   "Begin each review by introducing yourself as a Grammatical Error Reviewer. ",
)

In [ ]:
ethics_reviewer = AssistantAgent(
    name="EthicsReviewer",
    llm_config=llm_config,
    system_message="As an ethics specialist, your role is to evaluate content for ethical integrity "
                  "and identify any potential moral concerns. "
                  "Provide up to 3 specific, actionable recommendations to address ethical isues. "
                  "Ensure your feedback is concise and directly applicable. "
                  "Start each review by introducing yourself as an Ethics Reviewer.",
)

In [ ]:
meta_reviewer = AssistantAgent(
    name="MetaReviewer",
    llm_config=llm_config,
    system_message = "You are a meta reviewer, you aggregate and review "
                     "the work of other reviewers and give a final sugestion on the content. ",
)

###Orchestrating the nested chat

In [ ]:
def reflection_message(recipient, messags, sender, config):
  return f'''Review the following content.
        \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" :
            "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1,
    },
    {
        "recipient": grammatical_error_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" :
            "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}.",},
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" :
            "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1,
    }

]

In [ ]:
reviewer.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [ ]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):


      Write a concise but engaging blogpost about GPUs.
      Make sure the blogpost is within 100 words.
      

--------------------------------------------------------------------------------
[autogen.oai.client: 05-13 03:55:20] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Writer (to Reviewer):

Title: Unleashing GPU Power

Graphics Processing Units (GPUs) have revolutionized computing by handling massive parallel tasks at incredible speeds. Originally designed for rendering 3D graphics in gaming, modern GPUs now drive AI research, data science, cryptocurrency mining, and scientific simulations. Their thousands of cores process complex calculations simultaneously, slashing computation times and unlocking breakthroughs in deep learning, real-time ray tracing, and high-performance computing. With cloud providers and consumer markets embracing GPU acceleration, developers can harness unparalleled performance for everything from immersive virtual realities to groundbreaking medical discoveries. GPUs truly power the next frontier of innovation.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

*****************************

SEO_reviewer (to Reviewer):

SEO Reviewer

1. Target and Embed Long-Tail Keywords  
   - Research high-value phrases like “GPU acceleration for AI research,” “best GPUs for deep learning,” or “cloud GPU computing solutions.”  
   - Revise the title to include your primary keyword (e.g., “Unleashing GPU Power: GPU Acceleration for AI Research & High-Performance Computing”).  
   - Sprinkle these long-tail terms in at least one H2, the first 100 words, and naturally throughout the body (aim for a 1–2% keyword density).

2. Improve Content Structure and Readability  
   - Break the text into clear sections with H2/H3 headings (e.g., “H2: How GPUs Speed Up Deep Learning,” “H2: GPU Use Cases in Science & Industry,” “H3: Real-Time Ray Tracing Benefits”).  
   - Convert benefits into a bulleted list to boost scannability (e.g., “• Thousands of parallel cores… • Reduced computation times…”).  
   - Add a brief intro paragraph that sets user intent (“In this guide, you’ll learn how modern GPUs 


********************************************************************************
Starting a new chat....

********************************************************************************
Reviewer (to GrammaticalErrorReviewer):

Review the following content.    
        

 Title: Unleashing GPU Power

Graphics Processing Units (GPUs) have revolutionized computing by handling massive parallel tasks at incredible speeds. Originally designed for rendering 3D graphics in gaming, modern GPUs now drive AI research, data science, cryptocurrency mining, and scientific simulations. Their thousands of cores process complex calculations simultaneously, slashing computation times and unlocking breakthroughs in deep learning, real-time ray tracing, and high-performance computing. With cloud providers and consumer markets embracing GPU acceleration, developers can harness unparalleled performance for everything from immersive virtual realities to groundbreaking medical discoveries. GPUs truly power 

GrammaticalErrorReviewer (to Reviewer):

I am a Grammatical Error Reviewer. Here are the three most significant issues and how to fix them:

1. Adverb vs. adjective misuse  
   – Issue: “massive parallel tasks” implies the tasks are large rather than being executed in parallel.  
   – Fix: Change to “massively parallel tasks” to correctly use the adverb for manner.  
     Example:  
     Original: “…by handling massive parallel tasks at incredible speeds.”  
     Revised:  “…by handling massively parallel tasks at incredible speeds.”

2. Awkward pronoun-led sentence  
   – Issue: Beginning with “Their thousands of cores…” buries the true subject (GPUs) and can read as vague.  
   – Fix: Lead with the noun “GPUs” or use a prepositional phrase for clarity.  
     Example:  
     Original: “Their thousands of cores process complex calculations simultaneously…”  
     Revised:  “With thousands of cores, GPUs process complex calculations simultaneously…”

3. Overly long, comma-heavy sentenc


********************************************************************************
Starting a new chat....

********************************************************************************
Reviewer (to EthicsReviewer):

Review the following content.    
        

 Title: Unleashing GPU Power

Graphics Processing Units (GPUs) have revolutionized computing by handling massive parallel tasks at incredible speeds. Originally designed for rendering 3D graphics in gaming, modern GPUs now drive AI research, data science, cryptocurrency mining, and scientific simulations. Their thousands of cores process complex calculations simultaneously, slashing computation times and unlocking breakthroughs in deep learning, real-time ray tracing, and high-performance computing. With cloud providers and consumer markets embracing GPU acceleration, developers can harness unparalleled performance for everything from immersive virtual realities to groundbreaking medical discoveries. GPUs truly power the next f

EthicsReviewer (to Reviewer):

Ethics Reviewer:

The GPU overview is informative, but it omits several ethical considerations around resource use, supply chains, and dual-use. Here are three actionable recommendations to strengthen its ethical integrity:

1. Highlight Environmental Impact  
   • Acknowledge GPUs’ high power consumption—especially in crypto-mining and AI training—and suggest energy-efficient models or use of renewable energy.  
   • Recommend linking to carbon-offset programs or industry benchmarks (e.g., J per teraFLOP) so readers can compare sustainability metrics.

2. Address Responsible Sourcing  
   • Note that GPU production relies on minerals (e.g., cobalt) with potential links to child labor and conflict zones.  
   • Encourage vendors to adopt “conflict-free” certifications and publish their supply-chain audits, or to support fair-trade mineral initiatives.

3. Promote Ethical Use of GPU-Powered AI  
   • Advise developers to follow established AI ethics guidel


********************************************************************************
Starting a new chat....

********************************************************************************
Reviewer (to MetaReviewer):

Aggregate feedback from all reviewers and give final suggestions on the writing.
Context: 
{"Reviewer":"SEO Reviewer","Review":"1. Target and Embed Long-Tail Keywords:\n- Research high-value phrases like “GPU acceleration for AI research,” “best GPUs for deep learning,” or “cloud GPU computing solutions.”\n- Revise the title to include your primary keyword (e.g., “Unleashing GPU Power: GPU Acceleration for AI Research & High-Performance Computing”).\n- Sprinkle these long-tail terms in at least one H2, the first 100 words, and naturally throughout the body (aim for a 1–2% keyword density).\n\n2. Improve Content Structure and Readability:\n- Break the text into clear sections with H2/H3 headings (e.g., “H2: How GPUs Speed Up Deep Learning,” “H2: GPU Use Cases in Science & I

MetaReviewer (to Reviewer):

Below is a consolidated set of recommendations organized by focus area. Applying these changes will improve search visibility, clarity, and ethical rigor while keeping your prose tight and reader-friendly.

1. Structure & SEO  
• Title & Keywords – Revise the title to include your primary long-tail keyword (e.g., “Unleashing GPU Power: GPU Acceleration for AI Research & High-Performance Computing”). Target terms such as “GPU acceleration for AI research,” “best GPUs for deep learning,” or “cloud GPU computing solutions.”  
• Headings & Flow – Break into clear H2/H3 sections:  
  – H2: Introduction to GPU Acceleration  
  – H2: How GPUs Speed Up Deep Learning  
  – H2: GPU Use Cases in Science & Industry  
  – H3: Real-Time Ray Tracing Benefits  
• Intro & CTA – Add a 1–2 sentence intro stating user intent (“Discover how GPU acceleration can slash your AI training times…”) and a concluding call-to-action (“Explore our GPU buying guide” or “Download the free 

Writer (to Reviewer):

Title: Unleashing GPU Power: GPU Acceleration for AI Research & High-Performance Computing

## Introduction to GPU Acceleration  
Discover how GPU acceleration can slash AI training times by leveraging massively parallel cores.

## How GPUs Speed Up Deep Learning  
With thousands of cores, GPUs process calculations simultaneously:  
• Faster model training  
• Real-time data analysis  

## GPU Use Cases in Science & Industry  
• AI research & deep learning  
• Cloud GPU computing solutions  

### Real-Time Ray Tracing Benefits  
Experience lifelike graphics in gaming and simulations.  

Ready to turbocharge your projects? Explore our GPU buying guide.  
Choose energy-efficient models and conflict-free sourcing; follow AI ethics guidelines for responsible use.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (48676b34-0d8f-4617-9158-7ae9c3a570a1): Maximum turns (2) reached


In [ ]:
print(res.summary)

Title: Unleashing GPU Power: GPU Acceleration for AI Research & High-Performance Computing

## Introduction to GPU Acceleration  
Discover how GPU acceleration can slash AI training times by leveraging massively parallel cores.

## How GPUs Speed Up Deep Learning  
With thousands of cores, GPUs process calculations simultaneously:  
• Faster model training  
• Real-time data analysis  

## GPU Use Cases in Science & Industry  
• AI research & deep learning  
• Cloud GPU computing solutions  

### Real-Time Ray Tracing Benefits  
Experience lifelike graphics in gaming and simulations.  

Ready to turbocharge your projects? Explore our GPU buying guide.  
Choose energy-efficient models and conflict-free sourcing; follow AI ethics guidelines for responsible use.
